# HEP Jet assignment project - Data analysis and particle finding script

## Import essential packages.
---
* We will use [uproot](https://github.com/scikit-hep/uproot) packages to parse our .root file.
* The content of function `particle properties` and `jet properties` is defined in `particle_properties.py` and `jet_properties.py`.

In [141]:
import uproot
import pandas as pd 
import numpy as np 
from particle_properties_uproot import particle_properties  #import particle properties helper function from particle_properties.py
from jet_properties_uproot import jet_properties  #import jet properties helper function from jet_properties.py
import h5py

## Loading data, determine parameters, and assign variable
---

In [142]:
data  = uproot.open('./data/10K_event/tag_1_delphes_events.root')['Delphes']
#data.show()

particle = particle_properties(data)
jet = jet_properties(data)

Length = len(particle.event)
test_length = 10

PID_W_plus = 24 
PID_W_minus = -24
PID_DOWN = 1
PID_DOWN_VAR = -1
PID_UP = 2
PID_UP_BAR = -2
PID_STRANGE = 3
PID_STRANGE_BAR = -3
PID_CHARM = 4
PID_CHARM_BAR = -4
PID_BOTTOM = 5
PID_BOTTOM_BAR = -5
PID_TOP = 6
PID_TOP_BAR = -6

top_idx = np.zeros(len(particle.event))
top_daughter_idx_1 = np.zeros(len(particle.event))
top_daughter_pid_1 = np.zeros(len(particle.event))
top_daughter_idx_2 = np.zeros(len(particle.event))
top_daughter_pid_2 = np.zeros(len(particle.event))

top_bar_idx = np.zeros(len(particle.event))
top_bar_daughter_idx_1 = np.zeros(len(particle.event))
top_bar_daughter_pid_1 = np.zeros(len(particle.event))
top_bar_daughter_idx_2 = np.zeros(len(particle.event))
top_bar_daughter_pid_2 = np.zeros(len(particle.event))

parton_array = np.zeros([ len(particle.event) , 6, 7])


## Event selection 
---
1. Must contain:
    * At least 2 b tagged jet.
    * At least 6 jet exists.
    * For each jet, require |$\eta$| < 2.5 and $P_{T}$ > 25GeV. 

In [143]:
#Generate maker for each stage(event selection and jet selection.)
marker_event = []
marker_jet = []
marker_bjet = []

for i in range(len(particle.event)):
    marker_event.append(0)
    marker_jet.append(np.zeros([len(jet.pt[i])]))
    marker_bjet.append(np.zeros([len(jet.pt[i])]))


marker_event = np.asanyarray(marker_event)
marker_jet = np.asanyarray(marker_jet)
marker_bjet = np.asanyarray(marker_bjet)

print(type(marker_event), type(marker_jet))
print(marker_event.shape, marker_jet.shape)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(10000,) (10000,)
/usr/local/lib64/python3.6/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [144]:
#Mark which event pass the selection
# print("+-----------------------------------------------------------------------------------------------------+")
# print("Start event selection.")
# for i in range(len(particle.event)):
#     num_of_eta_in_range = np.sum(jet.eta[i] < 2.4 ) 
#     num_of_jet = len(jet.pt[i])
#     num_of_btagged = np.sum(jet.btag[i] == 1)
#     if num_of_eta_in_range >= 6 and num_of_jet >=6 and num_of_btagged >= 2: 
#         marker_event[i] = 1
#     else :
#         pass
# print("Event selection doen.")
# print("+-----------------------------------------------------------------------------------------------------+")

#Mark which jet in each event pass the selection.
print("+-----------------------------------------------------------------------------------------------------+")
print("Start jet selection.")
for i in range(len(particle.event)):
    for j in range(len(jet.pt[i])):
        if jet.btag[i][j] == 1 and jet.pt[i][j] > 25 and np.abs(jet.eta[i][j]) < 2.5:
            marker_bjet[i][j] = 1 
        elif jet.pt[i][j] > 25 and np.abs(jet.eta[i][j]) < 2.5:
            marker_jet[i][j] = 1

for i in range(len(particle.event)):
    if np.sum(marker_jet[i] == 1) >= 6 and np.sum(marker_bjet[i] == 1) >= 2 :
        marker_event[i] = 1 
print("Jet selection doen.")
print("+-----------------------------------------------------------------------------------------------------+")

+-----------------------------------------------------------------------------------------------------+
Start jet selection.
Jet selection doen.
+-----------------------------------------------------------------------------------------------------+


In [145]:
np.sum(marker_event == 1)

295

In [146]:
jet_pt = []
jet_eta = []
jet_phi = []
jet_mass = []
jet_btag = []

In [147]:
for i in range(len(jet.event)):
    if marker_event[i] == 1:
        jet_pt_tmp = []
        jet_eta_tmp = []
        jet_phi_tmp = []
        jet_mass_tmp = []
        jet_btag_tmp = []
        for j in range(len(jet.pt[i])):
            if marker_jet[i][j] == 1:
                jet_pt_tmp.append(jet.pt[i][j])
                jet_eta_tmp.append(jet.eta[i][j])
                jet_phi_tmp.append(jet.phi[i][j])
                jet_mass_tmp.append(jet.mass[i][j])
                jet_btag_tmp.append(jet.btag[i][j])
                
        jet_pt.append(jet_pt_tmp)
        jet_eta.append(jet_eta_tmp)
        jet_phi.append(jet_phi_tmp)
        jet_mass.append(jet_mass_tmp)
        jet_btag.append(jet_btag_tmp)

In [148]:
jet_eta[0]

[-1.1426754, -1.0607479, 0.010001938, -0.55345505, 2.3994348, 0.09344425]

## Particle tracing and daughter finding section
---

In [149]:
def shift_particle_tracing(dataset, PID_d, idx):
    if (dataset.iloc[idx,6] == PID_d):
        return dataset.iloc[idx,4]

def particle_tracing(dataset, PID, STATUS):

    for i in range(len(dataset)):
        if(dataset.iloc[i,1] == STATUS and dataset.iloc[i,6] == PID ): 
            daughter_index = int(dataset.iloc[i,0])
    if( dataset.iloc[daughter_index,6] == PID ):
        shifted_particle_index = dataset.iloc[daughter_index, 4]


    while dataset.iloc[shifted_particle_index,6] == PID:
            init_shifted_particle_index = shifted_particle_index
            shifted_particle_index = shift_particle_tracing(dataset, PID, init_shifted_particle_index)       

    dauthter_idx_1 = dataset.iloc[init_shifted_particle_index, 4]
    daughter_pid_1 = dataset.iloc[dauthter_idx_1, 6]

    dauthter_idx_2 = dataset.iloc[init_shifted_particle_index, 5]
    daughter_pid_2 = dataset.iloc[dauthter_idx_2, 6]

    return init_shifted_particle_index, dauthter_idx_1, daughter_pid_1, dauthter_idx_2, daughter_pid_2


In [150]:
for i in range(len(particle.event)):
    if marker_event[i] == 1:
        #print("+------------------------------------------------------------------------------------------------------+")
        #print("Start parsing event : {0}\nStart to trace top quark and find its daughters.".format(i))
        top_idx[i], top_daughter_idx_1[i], top_daughter_pid_1[i], top_daughter_idx_2[i], top_daughter_pid_2[i] = particle_tracing(particle.dataframelize(i), PID_TOP, 22)
        #print("+------------------------------------------------------~-----------------------------------------------+")
        #print("Start to find top_bar quark and its daughters.")
        top_bar_idx[i], top_bar_daughter_idx_1[i], top_bar_daughter_pid_1[i], top_bar_daughter_idx_2[i], top_bar_daughter_pid_2[i] = particle_tracing(particle.dataframelize(i), PID_TOP_BAR, 22)
        #print("+------------------------------------------------------------------------------------------------------+")

### Tracing the daughter 

In [151]:
#Input two daughter of top/top_bar and find their daughter
def quark_finder(dataset, mother_idx_1, mother_idx_2):
    
    #Specific two daughter of top
    def W_b_specifier(dataset, input_1_idx, input_2_idx):
        if dataset.iloc[int(input_1_idx),6] == PID_W_plus or dataset.iloc[int(input_1_idx),6] == PID_W_minus :
            return int(input_1_idx), int(dataset.iloc[int(input_1_idx),6]), int(input_2_idx)
        elif dataset.iloc[int(input_1_idx),6] == PID_BOTTOM or dataset.iloc[int(input_1_idx),6] == PID_BOTTOM_BAR :
            return  int(input_2_idx), int(dataset.iloc[int(input_1_idx),6]), int(input_1_idx)
        else :
            pass
            #print("Please check your data.")
    
    W_boson_idx, mother_pid, b_quark_idx = W_b_specifier(dataset, mother_idx_1, mother_idx_2)
    
    #Find the two daughters of boson
    
    daughter_1_idx = dataset.iloc[W_boson_idx, 4]
    daughter_1_pid = dataset.iloc[daughter_1_idx, 6]
    daughter_2_idx = dataset.iloc[W_boson_idx, 5]
    daughter_2_pid = dataset.iloc[daughter_2_idx, 6]

    
    if daughter_1_pid == mother_pid or daughter_2_pid == mother_pid:

        init_idx = W_boson_idx
        daughter_pid = daughter_1_pid
        if daughter_2_pid == mother_pid:
            daughter_pid = daughter_2_pid
        while daughter_pid == mother_pid :
            daughter_1_idx = dataset.iloc[int(init_idx), 4]
            daughter_2_idx = dataset.iloc[int(init_idx), 5]

            daughter_1_pid = dataset.iloc[int(daughter_1_idx), 6]
            daughter_2_pid = dataset.iloc[int(daughter_2_idx), 6]

            daughter_pid = daughter_1_pid
            init_idx = daughter_1_idx
            if daughter_2_pid == mother_pid:
                daughter_pid = daughter_2_pid
                init_idx = daughter_2_idx
            
            
            #print("Temporary daughter 1 indxe: {0}, PID: {1}".format(daughter_1_idx, daughter_1_pid))
            #print("Temporary daughter 2 indxe: {0}, PID: {1}".format(daughter_2_idx, daughter_2_pid))

    
    #print("Found daughter 1 index: {0}, PID: {1}.\nFound daughter 2 index: {2}, PID: {3}".format(daughter_1_idx, daughter_1_pid, daughter_2_idx, daughter_2_pid))
    return  b_quark_idx, daughter_1_idx, daughter_2_idx

In [152]:
for i in range(len(particle.event)):
    if marker_event[i] == 1 :
        #print("+------------------------------------------------------------------------------------------------------+")
        #print("Start parsing event : {0}\nStart to find top quark's daughters.".format(i))
        parton_array[i][0][0], parton_array[i][1][0], parton_array[i][2][0] = quark_finder(particle.dataframelize(i), top_daughter_idx_1[i], top_daughter_idx_2[i])
        #print("+------------------------------------------------------~-----------------------------------------------+")
        #print("Start to find top_bar quark's daughters.")
        parton_array[i][3][0], parton_array[i][4][0], parton_array[i][5][0], = quark_finder(particle.dataframelize(i), top_bar_daughter_idx_1[i], top_bar_daughter_idx_2[i])
        #print("+------------------------------------------------------------------------------------------------------+")
    elif marker_event[i] == 0 :
        parton_array[i] = 'Nan'
    else: pass

In [153]:
# ### 
# t t~ W+ W- b b~ 
# 0 0  0  0  0 0

# i.e.
# col 3 = 100010 
# col 5 = 101000
# col 6 = 101000
# col 8 = 010100
# col 10= 010001
# col 11= 010001

In [154]:
parton_pdgid = []
parton_barcode = []
parton_pt = []
parton_eta = []
parton_phi = []
parton_mass = []

In [155]:
barcode = np.array([34, 40, 40, 17, 20, 20])
for i in range(len(particle.event)):
    if marker_event[i] == 1:
        _parton_pdgid = []
        _parton_barcode = []
        _parton_pt = []
        _parton_eta = []
        _parton_phi = []
        _parton_mass = []
        for j in range(0,6):
            dataset = particle.dataframelize(i)

            _parton_pdgid.append(dataset.iloc[int(parton_array[i][j][0]), 6])
            _parton_barcode.append(barcode[j])
            _parton_pt.append(dataset.iloc[int(parton_array[i][j][0]), 7])
            _parton_eta.append(dataset.iloc[int(parton_array[i][j][0]), 8])
            _parton_phi.append(dataset.iloc[int(parton_array[i][j][0]), 9])
            _parton_mass.append(dataset.iloc[int(parton_array[i][j][0]), 10])

            # parton_array[i][j][1] = dataset.iloc[int(parton_array[i][j][0]), 6]  #PDGID
            # parton_array[i][j][2] = barcode[j]
            # parton_array[i][j][3] = dataset.iloc[int(parton_array[i][j][0]), 7]  #Pt
            # parton_array[i][j][4] = dataset.iloc[int(parton_array[i][j][0]), 8]  #Eta
            # parton_array[i][j][5] = dataset.iloc[int(parton_array[i][j][0]), 9]  #Phi
            # parton_array[i][j][6] = dataset.iloc[int(parton_array[i][j][0]), 10]  #Mass
        parton_pdgid.append(_parton_pdgid)
        parton_barcode.append(_parton_barcode)
        parton_pt.append(_parton_pt)
        parton_eta.append(_parton_eta)
        parton_phi.append(_parton_phi)
        parton_mass.append(_parton_mass)

## Parton-jet matching section
---

### Define the function for computing delta_R


In [156]:
def deltaPhi(phi1,phi2):
    phi = phi1-phi2
    while phi >= np.pi: phi -= np.pi*2.
    while phi < -np.pi: phi += np.pi*2.
    return phi

def delta_R(eta1, phi1, eta2, phi2):
    return np.sqrt(deltaPhi(phi1,phi2)**2+(eta1-eta2)**2)


In [157]:
dR_between_parton_jet = []
dR_between_parton_parton = []

for i in range(len(parton_pdgid)):
    dR_between_parton_jet.append(np.zeros([len(jet_pt[i]) * 6])) # # of connection = num of jet * num of parton
    dR_between_parton_parton.append(np.zeros([15])) # C^{6}_{2} = 15

dR_between_parton_jet = np.asanyarray(dR_between_parton_jet)
dR_between_parton_parton = np.asanyarray(dR_between_parton_parton)



In [158]:
max_num_of_jet_cand = []
for i in range(len(parton_pdgid)):
    max_num_of_jet_cand.append(len(jet_pt[i]))
max_num_of_jet_cand = np.asanyarray(max_num_of_jet_cand)
max_num_of_jet = max_num_of_jet_cand.max()
print(max_num_of_jet)

matching_jet = []
matching_parton = []
for i in range(len(parton_pdgid)):
    matching_jet.append(np.zeros([len(jet_pt[i])]))
    matching_parton.append(np.zeros([6]))

matching_jet = np.array(matching_jet)
matching_parton = np.array(matching_parton)

10
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


### Computing delta_R between each parton and jet

In [159]:
for i in range(len(parton_pdgid)):
    j = 0
    a = 0
    b = 0
    while a < 6 :
        for b in range( len(jet_pt[i]) ):
            #print(i, j)
            #print(i, a, b)
            #print(delta_R( parton_array[i][a][4], parton_array[i][a][5], jet.eta[i][b], jet.phi[i][b]))
            dR_between_parton_jet[i][j] = delta_R( parton_eta[i][a], parton_phi[i][a], jet_eta[i][b], jet_phi[i][b])
            j +=1
        a += 1 


In [160]:
dR_between_parton_jet[0]

array([1.90980065, 3.03773265, 1.51916387, 3.25589483, 2.84000266,
       0.98988872, 2.63036489, 0.6777005 , 1.71636584, 0.04152919,
       3.07765474, 2.5406051 , 2.3956489 , 1.57741739, 0.0158965 ,
       1.752638  , 2.52084275, 2.21257546, 3.83343412, 2.7999795 ,
       2.33277431, 2.25297774, 1.04526859, 2.9327267 , 4.52025844,
       3.42765233, 2.4612327 , 3.02988325, 0.08072635, 3.7454667 ,
       2.35760793, 3.37459311, 1.81671618, 3.50919071, 2.6068981 ,
       1.29019186])

### Matching jet and parton by finding the Min(dR(parton, jet))

In [161]:

for i in range(len(parton_pt)):
#for i in range(0,10):

    print("+------------------------------------------------------------------------------------------------------+")
    print(i)
    #print(dR_between_parton_jet.shape)
    array = np.reshape(dR_between_parton_jet[i], [6, len(jet_pt[i])])
    #print(array.shape)
        
    dataset = pd.DataFrame({'0': array[0,:], 
                                '1': array[1,:],
                                '2': array[2,:],
                                '3': array[3,:],
                                '4': array[4,:],
                                '5': array[5,:],
                                })
    print(dataset)

    for j in range(0,6):
        print("+------------------------------------------------------------------------------------------------------+")
        min_val = dataset.stack().min()
        if min_val < 0.4:
            print("Min val: {0}".format(min_val))
            min_idx, min_col = dataset.stack().idxmin()
            matching_parton[i][j] = int(min_idx)
            matching_jet[i][j] = int(min_col)
            print("The position of minimun appears. Raw: {0}, Colume: {1}".format(min_idx, min_col))
            dataset = dataset.drop([min_col], axis=1)
            dataset = dataset.drop([min_idx], axis=0)
            print("The dataset after delete the minimun's raw and colume:")
            print(dataset)
        else:
            matching_parton[i][j] = 'Nan'
            matching_jet[i][j] = 'Nan'
    for k in range(6, len(jet_pt[i])):
        matching_jet[i][k] = 'Nan'

        

+------------------------------------------------------------------------------------------------------+
0
          0         1         2         3         4         5
0  1.909801  2.630365  2.395649  3.833434  4.520258  2.357608
1  3.037733  0.677700  1.577417  2.799980  3.427652  3.374593
2  1.519164  1.716366  0.015896  2.332774  2.461233  1.816716
3  3.255895  0.041529  1.752638  2.252978  3.029883  3.509191
4  2.840003  3.077655  2.520843  1.045269  0.080726  2.606898
5  0.989889  2.540605  2.212575  2.932727  3.745467  1.290192
+------------------------------------------------------------------------------------------------------+
Min val: 0.015896495663126505
The position of minimun appears. Raw: 2, Colume: 2
The dataset after delete the minimun's raw and colume:
          0         1         3         4         5
0  1.909801  2.630365  3.833434  4.520258  2.357608
1  3.037733  0.677700  2.799980  3.427652  3.374593
3  3.255895  0.041529  2.252978  3.029883  3.509191
4  2.84000

In [162]:
for i in range(len(parton_pdgid)):
    print(i)
    array = np.reshape(dR_between_parton_jet[i], [6, len(jet_pt[i])])
    print(matching_parton[i], matching_jet[i])

0
[ 2.  3.  4. nan nan nan] [ 2.  1.  4. nan nan nan]
1
[ 1.  4.  3.  6. nan nan] [ 5.  4.  1.  2. nan nan nan]
2
[ 2.  1.  3.  4. nan nan] [ 4.  1.  2.  5. nan nan]
3
[ 1.  0.  4.  3. nan nan] [ 4.  1.  2.  0. nan nan]
4
[ 3.  2.  1. nan nan nan] [ 2.  4.  1. nan nan nan]
5
[ 1.  0.  2.  5. nan nan] [ 1.  2.  5.  4. nan nan]
6
[ 6.  4.  2.  5.  1. nan] [ 4.  5.  1.  2.  0. nan nan]
7
[ 0.  2.  1. nan nan nan] [ 5.  1.  2. nan nan nan]
8
[ 2.  3.  1.  4. nan nan] [ 1.  4.  5.  2. nan nan]
9
[ 0.  5.  1.  4. nan nan] [ 2.  5.  1.  4. nan nan]
10
[ 3.  2.  1.  4. nan nan] [ 2.  1.  5.  4. nan nan]
11
[ 2.  3.  1.  5.  4. nan] [ 5.  2.  0.  1.  4. nan]
12
[ 5.  1.  2. nan nan nan] [ 5.  2.  4. nan nan nan]
13
[ 1.  2.  3. nan nan nan] [ 5.  2.  4. nan nan nan]
14
[ 3.  1. nan nan nan nan] [ 1.  2. nan nan nan nan nan]
15
[ 3.  0.  2. nan nan nan] [ 1.  2.  5. nan nan nan]
16
[ 1.  3.  5.  0. nan nan] [ 5.  0.  2.  4. nan nan]
17
[ 1.  3.  5.  2.  0. nan] [ 2.  5.  1.  4.  3. nan]
18
[ 1. 

In [163]:
matching_parton[8]

array([ 2.,  3.,  1.,  4., nan, nan])

In [164]:
matching_jet[8]

array([ 1.,  4.,  5.,  2., nan, nan])

In [165]:
array = np.reshape(dR_between_parton_jet[8], [6, len(jet_pt[8])])
print(array[3,1])
dataset = pd.DataFrame({'0': array[0,:], 
                                '1': array[1,:],
                                '2': array[2,:],
                                '3': array[3,:],
                                '4': array[4,:],
                                '5': array[5,:],
                                })
print(dataset)
min_idx, min_col = dataset.stack().idxmin()
print(min_idx, min_col)
print(dataset.iloc[4, 5])

0.8024674005987997
          0         1         2         3         4         5
0  1.968971  1.290930  1.177561  3.028859  1.447221  2.477006
1  2.236573  2.803165  3.154249  0.802467  1.384669  0.119113
2  1.682753  0.040061  1.514747  3.141123  1.467601  2.883832
3  3.000730  1.408471  2.486073  1.779217  0.067842  1.460216
4  1.018997  1.694065  0.162071  3.304414  2.557589  3.224881
5  2.646979  2.515188  3.453835  0.711085  1.084008  0.444510
2 1
3.2248808500571395


In [166]:
parton_jet_index = np.zeros([len(parton_pdgid), 6])
jet_parton_index = []
np.zeros([len(parton_pdgid), 6])
for i in range(len(parton_pdgid)):
    jet_parton_index.append(np.zeros([len(jet_pt[i])]))


for i in range(len(parton_pdgid)):
    for j in range(0,6):
        parton_jet_index[i][j] = matching_parton[i][j]
    for k in range(len(jet_pt[i])):
        jet_parton_index[i][k] = matching_jet[i][k]

In [167]:
jet_barcode = []
for i in range(len(parton_pdgid)):
    jet_barcode.append(np.zeros([len(jet_pt[i])]))

jet_barcode = np.array(jet_barcode)

for i in range(len(parton_pdgid)):
    for j in range(len(jet_parton_index[i])):
        if jet_parton_index[i][j] == 0:
            jet_barcode[i][j] = barcode[0]
        elif jet_parton_index[i][j] == 1: 
            jet_barcode[i][j] = barcode[1]
        elif jet_parton_index[i][j] == 2: 
            jet_barcode[i][j] = barcode[2]
        elif jet_parton_index[i][j] == 3: 
            jet_barcode[i][j] = barcode[3]
        elif jet_parton_index[i][j] == 4: 
            jet_barcode[i][j] = barcode[4]
        elif jet_parton_index[i][j] == 5: 
            jet_barcode[i][j] = barcode[5]
        else :
            jet_barcode[i][j] = 'Nan'


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """


In [168]:
jet_barcode[8]

array([40., 20., 20., 40., nan, nan])

In [169]:
count_jet = 0
count_match_jet = 0
for i in range(len(jet_parton_index)):
    for j in range(len(jet_parton_index[i])):
        count_jet += 1
        if jet_parton_index[i][j] <= 5:
            count_match_jet +=1
        else : pass
print(count_jet, count_match_jet)

1910 1009


In [170]:
count_top = 0
count_top_bar = 0 
for i in range(len(jet_parton_index)):
    for j in range(len(jet_parton_index[i])):
        if np.sum(jet_parton_index[i] <= 2) == 3:
            #print(jet_parton_index[i])
            count_top +=1
        elif np.sum(jet_parton_index[i] > 2) == 3:
            #print(jet_parton_index[i])
            count_top_bar +=1
        else : pass
print(count_top, count_top_bar)

169 69


## Saved selected events
---

In [172]:
# lene = len(parton_pdgid)
# #Save the event which pass the selection
# with h5py.File("./event_record_pptt_10k_new.h5",'w') as f:
#     dt = h5py.vlen_dtype(np.dtype('int32'))

#     hdf5_jet_parton_index = f.create_dataset('jet_parton_index', (lene, ), dtype=dt)
#     hdf5_jet_barcode = f.create_dataset('jet_barcode', (lene, ), dtype=dt)
#     hdf5_jet_pt = f.create_dataset('jet_pt', (lene, ), dtype=dt)
#     hdf5_jet_eta = f.create_dataset('jet_eta', (lene, ), dtype=dt)
#     hdf5_jet_phi = f.create_dataset('jet_phi', (lene, ), dtype=dt)
#     hdf5_jet_mass = f.create_dataset('jet_mass', (lene, ), dtype=dt)
#     hdf5_jet_btag = f.create_dataset('jet_btag', (lene, ), dtype=dt)

#     for i in range(lene):
#         hdf5_jet_parton_index[i] = jet_parton_index[i]
#         hdf5_jet_barcode[i] = jet_barcode[i]
#         hdf5_jet_pt[i] = jet_pt[i]
#         hdf5_jet_eta[i] = jet_eta[i]
#         hdf5_jet_phi[i] = jet_phi[i]
#         hdf5_jet_mass[i] = jet_mass[i]
#         hdf5_jet_btag[i] = jet_btag[i]

#     hdf5_parton_jet_index = f.create_dataset('parton_jet_index', (lene, ), dtype=dt)
#     hdf5_parton_pdgid = f.create_dataset('parton_pdgid', (lene, ), dtype=dt)
#     hdf5_parton_barcode = f.create_dataset('parton_barcode', (lene, ), dtype=dt)
#     hdf5_parton_pt = f.create_dataset('parton_pt', (lene, ), dtype=dt)
#     hdf5_parton_eta = f.create_dataset('parton_eta', (lene, ), dtype=dt)
#     hdf5_parton_phi = f.create_dataset('parton_phi', (lene, ), dtype=dt)
#     hdf5_parton_mass = f.create_dataset('parton_mass', (lene, ), dtype=dt)

#     for i in range(lene):
#         hdf5_parton_jet_index[i] = parton_jet_index[i]
#         hdf5_parton_pdgid[i] = parton_pdgid[i]
#         hdf5_parton_barcode[i] = parton_barcode[i]
#         hdf5_parton_pt[i] = parton_pt[i]
#         hdf5_parton_eta[i] = parton_eta[i]
#         hdf5_parton_phi[i] = parton_phi[i]
#         hdf5_parton_mass[i] = parton_mass[i]
    